# Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Code IRIS (code quartier de l'INSEE)
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse

## Lecture du fichier DVF

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [3]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 0
row_last = 1000

In [4]:
# Chargement du jeu de données
df = pd.read_csv('valeursfoncieres-2020.txt',sep="|",decimal=",")
df.head()

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [5]:
print(df.columns)

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


## Constitution d'une base adresse à partir du fichier DVF

In [6]:
# focus on the address datas
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal','Commune', 'Code departement', 'Code commune']]
ad.shape

(2459560, 7)

In [7]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1,203
4,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203


In [8]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1059450, 7)

In [9]:
# Traitement des codes postaux (transformés en INT sauf les NA)

ad['Code postal'] = ad['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289


In [10]:
ad['Code departement'] = ad['Code departement'].apply(lambda x: str(x).zfill(2) if len(str(x)) == 1 else x)
ad['Code commune'] = ad['Code commune'].apply(lambda x: str(x).zfill(3) if len(str(x)) < 3 else x)
ad['Code commune INSEE'] = ad['Code departement'].astype(str) + ad['Code commune'].astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289


In [11]:
# On formate les adresses
# tools functions : nan values management
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [12]:
# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
ad['Ville'] = ad['Code postal'] + " " + ad['Commune']
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS


In [13]:
# Création du champ "addressName" qui sera utilisée dans la requête API
ad['Adresse'] = ad.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [14]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [15]:
ad['Ville'] = ad['Ville'].replace('-', ' ', regex=True).astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


## Sélection des adresses à parser pour ajout des coordonnées GPS et codes IRIS

In [16]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = ad.iloc[row_first:row_last, :]
selec.shape

(1000, 10)

In [17]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes (nécessaire pour code IRIS)
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [18]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [19]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [20]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,"46.1832170988,5.32007207542"
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,"46.2469919709,4.89933490972"
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,"46.2469919709,4.89933490972"
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,"46.2469919709,4.89933490972"
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,"46.1711903413,5.21802169564"


In [21]:
# Test sur Marseille car les codes communes INSEE semblent être différents selon la source
selec[(selec['Ville'].str.contains("MARSEILLE")) & (selec['Code departement'] == 13)]

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps


In [22]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,46.1832170988,5.32007207542
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,46.2469919709,4.89933490972
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,46.2469919709,4.89933490972
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,46.2469919709,4.89933490972
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,46.1711903413,5.21802169564
5,NaN,NaN,CHAMP PORTIER,1370,VAL-REVERMONT,01,426,01426,1370 VAL REVERMONT,CHAMP PORTIER 1370 VAL-REVERMONT,NaN,NaN,NaN
6,NaN,NaN,CHAMPEL,1270,COLIGNY,01,108,01108,1270 COLIGNY,CHAMPEL 1270 COLIGNY,01108,46.3866637865,5.3308227014
7,5367.0,NaN,LORETTE,1270,COLIGNY,01,108,01108,1270 COLIGNY,5367 LORETTE 1270 COLIGNY,01108,46.3866637865,5.3308227014
8,31.0,RUE,COMTE DE LA TEYSSONNIERE,1000,BOURG-EN-BRESSE,01,053,01053,1000 BOURG EN BRESSE,31 RUE COMTE DE LA TEYSSONNIERE 1000 BOURG-EN-...,01053,46.2051520382,5.24602125501
9,6.0,AV,DU CHAMP DE FOIRE,1000,BOURG-EN-BRESSE,01,053,01053,1000 BOURG EN BRESSE,6 AV DU CHAMP DE FOIRE 1000 BOURG-EN-BRESSE,01053,46.2051520382,5.24602125501


In [23]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,46.1832170988,5.32007207542,,,
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,46.1711903413,5.21802169564,,,


In [24]:
selec.shape

(1000, 16)

## Récupération des codes IRIS et coordonnées GPS à partir de l'API Nominatim et geo.api.gouv

In [26]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="pyPredImmo")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")
n=0

for row in range(row_first, row_last):
    
    iris = ""
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""

    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]

    # Interrogation de l'API Nominatim
    try:
        location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 5)
    
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    if location is None:                      # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude_api == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS pour code IRIS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            try:
                result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
                                 + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            except requests.exceptions.RequestException as e: 
                raise SystemExit(e)
            
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude_adresse) + "," + str(longitude_adresse) + " (code IRIS = " + str(iris) + ")")
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude_adresse
    selec['lat'][row] = latitude_adresse
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        etape = time.time()

    if (n % 10000 == 0):
        selec.iloc[(n - 10000) : n, :].to_csv(
            'export_gps_iris_' + str(row_first + n - 10000) + "-" + str(row_first + n)+'.csv')
        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
13:16:53


FORTUNAT 1250 CEYZERIAT ===> , (code IRIS = 010720000)
EN COROBERT 1290 LAIZ ===> , (code IRIS = 012030000)
RUE DE LA CHARTREUSE 1960 PERONNAS ===> , (code IRIS = 012890102)
CHAMP PORTIER 1370 VAL-REVERMONT ===> , (code IRIS = 014260000)
CHAMPEL 1270 COLIGNY ===> , (code IRIS = 011080000)
5367  LORETTE 1270 COLIGNY ===> , (code IRIS = 011080000)
31 RUE COMTE DE LA TEYSSONNIERE 1000 BOURG-EN-BRESSE ===> , (code IRIS = 010530203)
6 AV DU CHAMP DE FOIRE 1000 BOURG-EN-BRESSE ===> , (code IRIS = 010530101)
LES CADALLES 1000 SAINT-DENIS-LES-BOURG ===> , (code IRIS = 013440000)
LES FOSSES 1440 VIRIAT ===> , (code IRIS = 014510101)
5104  COLIGNY LE BAS 1270 COLIGNY ===> , (code IRIS = 011080000)
16 RUE DES GRAVES 1000 BOURG-EN-BRESSE ===> , (code IRIS = 010530302)
2 IMP DU COLOMBIER 1250 TOSSIAT ===> , (code IRIS = 014220000)
102 IMP DE LA CULOTTE 1310 MONTRACOL ===> , (code IRIS = 012640000)
LE VILLAGE 1310 MONTRACOL ===> , (code IRIS = 012640000)
10 ALL DE LA PETITE

KeyboardInterrupt: 

In [27]:
geolocator.geocode("FORTUNAT 1250 CEYZERIAT", country_codes = 'fr', timeout = 5)

Location(Chemin de Fortunat, Ceyzériat, Bourg-en-Bresse, Ain, Auvergne-Rhône-Alpes, France métropolitaine, 01250, France, (46.1737528, 5.3230074, 0.0))

In [ ]:
selec.head()

In [ ]:
selec.shape

## Export de la totalité de la sélection d'adresses après la fin du job

In [ ]:
selec.to_csv('export_gps_iris_' + str(row_first) + "-" + str(row_last)+'.csv')